## 销量变化
由于天猫只显示从本天以前的30天内的总销量，销量的数字是非常难处理的，我们对数字处理如下。
+ 首先对销量加1取log值，避免0值报错。
+ 再用本天减去上一天的销量，那么会得到，这一天比上个月同日期的多出来的销量。
+ 我们假设上个月的这些天，因为没有平台活动，销量是平稳的。
+ 那么计算结果，可以表现每天的销量变化

### 总的销量变化
![img](https://github.com/codeandworld/photo/blob/master/double11_data_bigquatity.png?raw=true)
+ 从图中我们可以看出，天猫官方更新销量数据是有延迟的，11-13的数据实际上反映的是11-11的销售量。
+ 11-11和11-12的销量出现非常大的增加，说明平台活动显著刺激了消费
+ 11-11和11-12前后的销量都比上月少，说明平台活动日抑制了活动日前后的消费
### 八大类销量变化
![img](https://github.com/codeandworld/photo/blob/master/double11_data_totalquatity.png?raw=true)
+ 从图中可以看出，销量增加最多的是衣服，教育娱乐产品

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Dec 03 14:41:00 2016

@author: Administrator
"""

import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt

# read document as dataframe
table = pd.read_csv(r'C:\Users\Administrator\Desktop\double11\our')
table = table[table['date']!='2016-11-04']

# get basket of goods
table_basket = table.iloc[:,[1,2,3,6]]
table_basket = table_basket[table_basket['date']=='2016-11-05']
table_basket = table_basket.iloc[:,[0,2,3]]
table_basket = table_basket.groupby(['catid', 'goodsID'], as_index=False).mean()
table_basket = table_basket.drop_duplicates(subset='goodsID')
table_basket = table_basket.set_index(['goodsID'])
table_basket = table_basket.drop(['monthly_sales'],1)


# get quatity of goods
table_quatity = table.iloc[:,[2,3,6]]
table_quatity = table_quatity.groupby(['date', 'goodsID'], as_index=False).mean()
table_quatity = pd.pivot_table(table_quatity, index=['goodsID'], values=['monthly_sales'], columns=['date'])
table_quatity.columns = ['11-05',
                        '11-06','11-07','11-08',
                        '11-09','11-10','11-11',
                        '11-12','11-13','11-14',
                        '11-15','11-16']

# connect quatity and basket of goods
quatity_change = pd.concat([table_basket, table_quatity], axis=1, join='inner')
quatity_change = quatity_change.drop(['11-14'],1)
for i in range(1,12):
    quatity_change.iloc[:,i] = quatity_change.iloc[:,i].apply(lambda x: math.log(x+1))
for i in range(11,1,-1):
    quatity_change.iloc[:,i] = quatity_change.iloc[:,i]-quatity_change.iloc[:,i-1]
quatity_change = quatity_change.drop(['11-05'],1)

# get small category mean quatity
small_total = quatity_change.groupby(['catid'], as_index=False).mean()
small_total['medium_catid'] = small_total['catid'].apply(lambda x: str(x)[0:6])
# get medium category mean quatity
medium_total = small_total.groupby(['medium_catid'], as_index=False).mean()
medium_total['big_catid'] = medium_total['medium_catid'].apply(lambda x: str(x)[0:4])
# get big category mean quatity
big_total = medium_total.groupby(['big_catid'], as_index=False).mean()

# big quatity change
big_total = big_total.drop(['catid'], 1)
big_total = big_total.set_index(['big_catid']) 
# total quatity change
total_basket = pd.DataFrame({'weight':[0.3139, 0.0399, 0.0851, 0.0584, 
                0.0904, 0.0925, 0.1415, 0.1782]}, 
                index = ['1101', '1102', '1103', '1104', '1105', '1106', '1107', '1108'])
total_basket = pd.concat([total_basket, big_total], axis=1)
for i in range(1,11):
    total_basket.iloc[:,i] = total_basket['weight']*total_basket.iloc[:,i]
total_basket = total_basket.drop(['weight'], 1) 
total_basket = total_basket.T
total_basket['total_index'] = total_basket.apply(lambda x: x.sum(), axis=1)
total_index = total_basket['total_index']

# plot totalindex
total_index.plot()
plt.savefig(r'C:\Users\Administrator\Desktop\double11_data_bigquatity.png')
# plot big_total
big_total = big_total.T
fig, axes = plt.subplots(nrows=4, ncols=2, sharex=True, sharey=True)
big_total['1101'].plot(ax=axes[0,0]); axes[0,0].set_title('food')
big_total['1102'].plot(ax=axes[0,1]); axes[0,1].set_title('tobacco and liquor')
big_total['1103'].plot(ax=axes[1,0]); axes[1,0].set_title('clothing')
big_total['1104'].plot(ax=axes[1,1]); axes[1,1].set_title('household')
big_total['1105'].plot(ax=axes[2,0]); axes[2,0].set_title('healthcare')
big_total['1106'].plot(ax=axes[2,1]); axes[2,1].set_title('transportation and communication')
big_total['1107'].plot(ax=axes[3,0]); axes[3,0].set_title('education and recreation')
big_total['1108'].plot(ax=axes[3,1]); axes[3,1].set_title('residence')
plt.savefig(r'C:\Users\Administrator\Desktop\double11_data_totalquatity.png')
